In [4]:
#from pymystem3 import Mystem
#!pip install pymorphy2
import pymorphy2
import re 
#!pip install pymystem3
#!pip install git+https://github.com/nlpub/pymystem3

In [3]:
class TextStorage():
    
    def __new__(self):
        if not hasattr(self, 'instance'):
            self.inst = super(TextStorage, self).__new__(self)
        return self.inst
    
    def __init__(self):
        
        self.__Text = { 'text': None ,'num_words' : 15 , 'words_list' : [],
                        'words_len': [], 'num_sents':None, 'sent_list':None,
                     'num_syms':None}   # Метаинформация о тексте
        
        self.__Sentence = {'simp_sent_list':[], 'sent_type':[], 'first_simp_sent_ind':[],
                          'num_words_simp_sent':[]
                            } # Информация о сложном предложении
        
        self.__SimpleSentence = {'word_comb':[], 'word':[], 'simp_sent_type':[]
                                } # Инф о простом предложении
        
        self.__WordCombination = {'first_word_ind':[],'word_comb_type':[],
                                  'main_word_ind':[]
                                 } # Инф о словосочетаниях
        
        self.__Word = {'gramm_class':[],'flect_class':[],'ending_len':[],
                      '4':[], '5':[]} # Инф о словах

    @property
    def Text(self):   
        return self.__Text   
    @Text.setter
    def Text(self, value):
        self.__Text = value
#        if a in self.__Text[a]:
#            print('da')
#            self.__Text[a] = value
#        else:
#            print('no attr')
    
    @property
    def Sent(self):   
        return self.__Sentence
    @Sent.setter
    def Sent(self, value):   
        self.__Sentence = value
        
    @property
    def SimpSent(self):   
        return self.__SimpleSentence
    @SimpSent.setter
    def SimpSent(self, value):   
        self.__SimpleSentence = value

    @property
    def WordComb(self):   
        return self.__WordCombination
    @WordComb.setter
    def WordComb(self, value):   
        self.__WordCombination = value

    @property
    def Word(self):   
        return self.__Word
    @Word.setter
    def Word(self, value):   
        self.__Word = value

In [88]:
storage = TextStorage()
storage.Text['raw_text']
storage.Text['raw_text'] = 'Машина ехала по дороге. Вася пил пиво'

In [85]:
storage.Text['raw_text']

'Машина ехала по дороге. Вася пил пиво'

In [90]:
storage.Text['num_words'] = 100

In [91]:
storage.Text['num_words']

100

In [6]:
#Функция нахождения словосочетаний по ключевым словам (на входе слова по классам)
def Find_Collocations(sentence):
    separators = ['F','&','V','L','K',',']
    posible_words = ['N','A','0','O','i','g','s','k','K','L','V','F','h','M','I']
    result = [[]]
    for sent in sentence:
        if (sent in separators) or not(sent in posible_words):
            result.append([sent])
            result.append([])
        else:
            result[-1].append(sent)
    i = 0
    while i < len(result):
        if len(result[i]) > 1:
            result[i] = tuple(result[i])
            i+=1
        elif len(result[i]) == 1:
            if not(result[i][0] in ['.',',']):
                result[i] = 'I'
            else:
                result[i] = result[i][0]
            i+=1
        else:
            del result[i]
    return result
#Функция определения типа словосочетания
def CollocationType(collocation):
    NA = ['N','A','0','O','i','g','s','k']
    VA = ['K','L','V','F','h','M','I']
    for word_type in collocation:
        if word_type in NA:
            return 'NA'
        elif word_type in VA:
            return 'VA'
def find_the_main_word(collocation):
    if len(collocation) == 1:
        return 1
    if CollocationType(collocation) == 'NA':
        if 'N' in collocation:
            return collocation.index('N') + 1
    elif CollocationType(collocation) == 'VA':
        if 'K' in collocation:
            return collocation.index('K') + 1
        elif 'L' in collocation:
            return collocation.index('L') + 1
        elif 'V' in collocation:
            return collocation.index('V') + 1
#Функция вывода информации о совосочетаниях (принимает на вход результат функции Collocations)
def CollocationsInfo(collocations):
    collocations_info = [[1,len(collocations[0])]]
    for encoding in collocations[1:]:
        
        collocations_info.append([collocations_info[-1][0]+collocations_info[-1][1],len(encoding)])
    for i in range(len(collocations)):
        collocations_info[i].append(CollocationType(collocations[i]))
        collocations_info[i].append(find_the_main_word(collocations[i]))
        collocations_info[i].append(collocations[i])
    k = 0 
    while k < len(collocations_info):
        if collocations_info[k][1] == 1:
            del collocations_info[k]
        else:
            k+=1
    return [tuple(info) for info in collocations_info]


#функция информация по словосочетаниям
def Collocations(sent, morf):
    colloc = Find_Collocations(sent)
    colloc_info = CollocationsInfo(colloc)
    for info in colloc_info:
        print('словосочетание:',info[4])
        print('№ первого слова словосочетания в предложении:',info[0])
        print('длинна словосочетания:', info[1])
        print('Тип словосочетания:', info[2])
        print('номер главного слова:',info[3])
        print()
    return colloc_info

In [7]:
#функция определения класса слов
def class_word(word_list):
    dict_class = {'NOUN':'N',
                    'ADJF':'A',
                    'VERB':'V',
                    'PRTS':'K',
                    'INFN':'I',
                    'PRTF':'W',
                    'GRND':'D',
                    'NUMR':'O',
                    'ADVB':'Y',
                    'PREP':'F',
                    'CONJ':'&',
                    'NPRO':'i',
                    'INTJ':'!'}
    morph = pymorphy2.MorphAnalyzer()
    list_punctuation =['.',',','!','–','"','?','(',')','{','}','[',']']
    sens = [morph.parse(word)[0].tag.POS if not(word in list_punctuation) 
            else word  for word in word_list]
    for i,word in enumerate(word_list):
        if (sens[i]=='VERB')and(morph.parse(word)[0].tag.tense=='past'):
            sens[i] = 'L' 
    list_class = [dict_class.get(word) if not(str(word) in list_punctuation) and (word in dict_class.keys())
            else word  for word in sens]
    return list_class

In [8]:
#функция нахождения главных слов в словосочетаниях
def Find_main_word(sent):
    coll_info = CollocationsInfo(Find_Collocations(sent))
    result = []
    for info in coll_info:
        result.append(info[0]+info[3]-2)
    return result

In [9]:
def find_all_N_morf_info(sent, morf):
    main_N = Find_main_word(sent)
    result = [(sent[N],N,morf[N]) for N in main_N]
    return result
def find_all_KLV_morf_info(sent, morf):
    result = []
    for i in range(len(sent)):
        if sent[i] in ['K', 'L', 'V']:
            result.append((sent[i], i, morf[i]))
    return result
#3 и далее. вторая равна, 0 у N,  3 у V
def equal(N, KLV):
    N_morf = N[2].split('/')[2:]
    KLV_morf = KLV[2].split('/')[2:]
    for morf_N_i in N_morf:
        for morf_KLV_i in KLV_morf:
            if KLV[0]=='V' and (morf_N_i[1]==morf_KLV_i[1]) and (morf_N_i[-1]=='0')and(morf_KLV_i[-1]=='3'):
                return True
            elif (KLV[0] in ['K','L']) and (morf_N_i[:2]==morf_KLV_i[:2]):
                return True
    return False

In [10]:
#функция определения подлежащего и сказуемого
def s_p(sent,morf):
    ind,ind_P=None,None
    sent1=sent.copy()
    if 'i' in sent:
        ind = sent.index('i')
        if ('V' in sent):
            ind_P = sent.index('V')
        elif('K' in sent):
            ind_P = sents.index('K')
        elif('L' in sent[ind:]):
            ind_P = sent.index('L')  
        sent1[ind]='S'
        sent1[ind_P]='P'
        return (sent1,ind,ind_P)
    possible_SP=[]
    all_N = find_all_N_morf_info(sent, morf)
    all_KLV = find_all_KLV_morf_info(sent, morf)
    for N in all_N:
        for KLV in all_KLV:
            if equal(N,KLV):
                possible_SP.append((N[1],KLV[1]))
    true_sp = False
    for SP in possible_SP:
        if SP[0]<SP[1]:
            true_sp = True
    if true_sp:
        i = 0
        while i < len(possible_SP):
            if possible_SP[i][0]>possible_SP[i][1]:
                del possible_SP[i]
            else:
                i+=1
    possible_SP[0][0]
    sent1[possible_SP[0][0]]='S'
    sent1[possible_SP[0][1]]='P'
    return (sent1,possible_SP[0][0],possible_SP[0][1])
    #return possible_SP

In [11]:
#функция определения скелета предложения
def skel_sent(sent_sp, ind_main, S,P):
    sent = sent_sp.copy()
    for i,k in enumerate(sent):
        if not((k =='F') or (i in ind_main+[S,P])or k in[',','.','–',':',';']):
            sent[i] = None
    return sent

In [12]:
def storage_interface(sent, TextStorage):
    morf_class = class_word(sent)
    colloc = Find_Collocations(morf_class)
    colloc_info = CollocationsInfo(colloc)
    first_word_ind = [info[0] for info in colloc_info]
    word_comb_type = [info[2] for info in colloc_info]
    main_word_ind = [info[3] for info in colloc_info]
    return {'first_word_ind':first_word_ind, 'word_comb_type':word_comb_type, 'main_word_ind':main_word_ind}

In [13]:
sent = "Сняв влажное покрывало со скульптуры в натуральную величину, он оценил взглядом свою работу."
sent = sent.split()
sent[7]='величину'
sent.insert(8,',')
sent[-1] = 'работу'
sent.insert(14,'.')

morf = ['00/152/0000','02/103/3110/3140','01/070/','00/160/0050','01/056/2120/2210/2240', '00/157/0040',
       '02/103/2140','01/056/2140','00/145/1110','00/117/1100','02/001/1150','01/103/2140','01/056/2140']
morf.insert(9,None)
morf.insert(10,None)
morf.insert(13,None)
morf.insert(15,None)

#классы слов
morf_klass = class_word(sent)
#morf_klass[9]='M'
#morf_klass[12]='M'
morf_klass

['D', 'A', 'N', 'F', 'N', 'F', 'A', 'N', ',', 'i', 'L', 'N', 'A', 'N', '.']

In [22]:
morf = ['00/152/0000','02/103/3110/3140','01/070/','00/160/0050','01/056/2120/2210/2240', '00/157/0040',
       '02/103/2140','01/056/2140','00/145/1110','00/117/1100','02/001/1150','01/103/2140','01/056/2140']
morf.insert(9,',')
morf.insert(15,'.')



In [27]:
#информация о словосочетаниях
print(Find_Collocations(morf_klass))
Collocations(morf_klass, morf)

['I', ('A', 'N'), 'I', 'I', 'I', ('A', 'N'), ',', 'I', 'I', ('N', 'A', 'N'), '.']
словосочетание: ('A', 'N')
№ первого слова словосочетания в предложении: 2
длинна словосочетания: 2
Тип словосочетания: NA
номер главного слова: 2

словосочетание: ('A', 'N')
№ первого слова словосочетания в предложении: 7
длинна словосочетания: 2
Тип словосочетания: NA
номер главного слова: 2

словосочетание: ('N', 'A', 'N')
№ первого слова словосочетания в предложении: 12
длинна словосочетания: 3
Тип словосочетания: NA
номер главного слова: 1



[(2, 2, 'NA', 2, ('A', 'N')),
 (7, 2, 'NA', 2, ('A', 'N')),
 (12, 3, 'NA', 1, ('N', 'A', 'N'))]

In [29]:
morf_klass2 = morf_klass.copy()
s_p(morf_klass2,morf)
#получение предложения с подлежащим и сказуемым. Вывод индекса подл. и сказ.
s_p_sent,S,P = s_p(morf_klass2,morf)
s_p_sent

['D', 'A', 'N', 'F', 'N', 'F', 'A', 'N', ',', 'S', 'P', 'N', 'A', 'N', '.']

In [33]:
#поиск индексов главных слов в словосочетаниях
ind_m = Find_main_word(morf_klass2)
#скелет предложения
print(skel_sent(s_p_sent, ind_m, S,P))

[None, None, 'N', 'F', None, 'F', None, 'N', ',', 'S', 'P', 'N', None, None, '.']


## Пример 2

In [67]:
sent2='Я устал от нашего жалкого существования .'
sent2_lis=sent2.split()
morf_klass = class_word(sent2_lis)


In [68]:
#классы слов
morf_klass

['i', 'L', 'F', 'A', 'A', 'N', '.']

In [69]:
#информация о словосочетаниях
morf=[]
Collocations(morf_klass, morf)

словосочетание: ('A', 'A', 'N')
№ первого слова словосочетания в предложении: 4
длинна словосочетания: 3
Тип словосочетания: NA
номер главного слова: 3



[(4, 3, 'NA', 3, ('A', 'A', 'N'))]

In [73]:
#поиск индексов главных слов в словосочетаниях
ind_m = Find_main_word(morf_klass)
#получение предложения с подлежащим и сказуемым. Вывод индекса подл. и сказ.
s_p_sent,S,P = s_p(morf_klass,morf)
s_p_sent

['S', 'P', 'F', 'A', 'A', 'N', '.']

In [74]:
#скелет предложения
skel_sent(s_p_sent, ind_m, S,P)

['S', 'P', 'F', None, None, 'N', '.']

In [50]:
s = 'Он идет гулять в парк'
s2=s.split()
morf_klass3 = class_word(s2)
print("классы слов:\n"+' '.join(morf_klass3))
#поиск индексов главных слов в словосочетаниях
ind_m = Find_main_word(morf_klass)
#получение предложения с подлежащим и сказуемым. Вывод индекса подл. и сказ.
s_p_sent,S,P = s_p(morf_klass,morf)
print("предложение с подлежащим:\n"+ " ".join(s_p_sent))
print("скелет предложения:\n")
skel_sent(s_p_sent, ind_m, S,P)

классы слов:
i V I F N
предложение с подлежащим:
S P F A A N .
скелет предложения:



['S', 'P', 'F', None, None, 'N', '.']

In [75]:
sent = "Параметры модели задаются либо статическими характеристиками , либо множествами возможных значений"
s2=sent.split()
s2_m=class_word(s2)
morf = ['01/001/1210/1240','01/055/2120/2130/2160/2210/2240','04/116/0203','00/156/0000','03/106/0250', '03/060/2250',
       ',','00/156/0000','03/070/3250','03/103/0220/0260','02/077/3220']

In [53]:
01/001/1210/1240
01/055/2120/2130/2160/2210/2240'

играли
род числ падеж лицо
00/125/1200/2200/3200

['Параметры',
 'модели',
 'задаются',
 'либо',
 'статическими',
 'характеристиками',
 ',',
 'либо',
 'множествами',
 'возможных',
 'значений']

In [5]:
s = TextStorage()

In [14]:
sent = "Параметры модели задаются либо статическими характеристиками , либо множествами возможных значений"
s2=sent.split()
storage_interface(s2)

{'first_word_ind': [1, 5, 9],
 'word_comb_type': ['NA', 'NA', 'NA'],
 'main_word_ind': [1, 2, 1]}

In [77]:
s_p(s2_m,morf)

(['S', 'N', 'P', '&', 'A', 'N', ',', '&', 'N', 'A', 'N'], 0, 2)